# The Intermediate 

The following notebook contains the code for the Intermediate strategy, aiming to obtain a medium level AI player, able to evaluate some interesting caracteristics of the state of the game and select the returning move at each turn.

For more details about this strategy, we recommend to check the slides.

## Import setup

In [2]:
import itertools

# Defining variables
The dictionary *values* represent the value associated to each card. The first element of the tuple stands for the rank of the card (from 1 to 10), while the second for its suit (1=coins, 2=cups, 3=swords, 4=clubs). 
To define the meaning of a card to be important, we followed some guidelines: we assigned more points to the cards which counts the most when counting the 'Primiera', then we gave bonus points to coins cards, because they are important for the 'Coins' counting, and finally we assigned a special value to number 7 and 10 of coins, because both 'Sette bello' and 'Re bello' make you gain a point.

In [3]:
values = {(1, 1): 26, (2, 1): 22, (3, 1): 23, (4, 1): 24, (5, 1): 25, (6, 1): 28, (7, 1): 139, (8, 1): 20, (9, 1): 20, (10, 1): 139,
          (1, 2): 16, (2, 2): 12, (3, 2): 13, (4, 2): 14, (5, 2): 15, (6, 2): 18, (7, 2): 29, (8, 2): 10, (9, 2): 10, (10, 2): 10,
          (1, 3): 16, (2, 3): 12, (3, 3): 13, (4, 3): 14, (5, 3): 15, (6, 3): 18, (7, 3): 29, (8, 3): 10, (9, 3): 10, (10, 3): 10,
          (1, 4): 16, (2, 4): 12, (3, 4): 13, (4, 4): 14, (5, 4): 15, (6, 4): 18, (7, 4): 29, (8, 4): 10, (9, 4): 10, (10, 4): 10}


# First def: CheckForScopa

Confront every card in the hand with the sum of the cards in the table: if they have the same value, save that card in a list.


Check the list to see if you can do a Scopa with more than one card. If so, play the card with the highest value.

In [4]:
def CheckForScopa(legalMoves, table, Standalone, values = values): # check if it's possible to do a scopa
    best_broom = None
    max_value = float('-inf')
    brooms = []
    for card in legalMoves:
        if card[0] == sum(rank_table[0] for rank_table in table):
            brooms.append(card)
        else:
            pass
    
    if len(brooms) == 0:
        return False
    else:
        for broom in brooms:
            if broom in values:
                value = values[broom]
                if value > max_value:
                    max_value = value
                    best_broom = broom
        if Standalone:
            return best_broom
        else:
            return {
                best_broom: table
            }

# Second def: AvoidScopa

For each card in the hand, it checks what would remain on the table if that card would be played:
It first checks all the 'single picking' cards, then the combinations of cards that it may take playing a single card: if the sum of the remaining cards is equal to a card that the next player might have (represented by the 'deck' variable), it doesn't play that card.
If none of the cards could pick anything from the table, or if taking anything would give the opponent a chance to make a Scopa, the code checks the best card to add to the table avoiding the following player to make a Scopa.
In case the next player could potentially make Scopa with any play, the code returns the card with the lowest value to decrease the loss of points.

In [5]:
def AvoidScopa(legalMoves, table, deck):
    list_table = [rank[0] for rank in table]
    list_deck = [rank[0] for rank in deck]
    possible_picks = {}
    possible_plays = []
    no_plays = []
    for card in legalMoves:
        if card[0] in list_table:
            copy = list_table[:]
            sum_table = sum(rank_table for rank_table in copy) - card[0]
            if sum_table in list_deck:
                pass
            else:
                possible_plays.append(card)
                for pick in table:
                    if pick[0] == card[0]:
                        possible_picks[card] = pick
                        break  
        else:
            copy = list_table[:]
            copy.append(card[0])
            sum_table = sum(rank_table for rank_table in copy)
            if sum_table in list_deck:
                pass
            else:
                possible_plays.append(card)
            subsets = []
            for r in range(2, len(list_table) + 1):
                for subset in itertools.combinations(list_table, r):
                    if sum(subset) == card[0]:
                        copy = list_table[:]
                        sum_table = sum(rank_table for rank_table in copy) - card[0]
                        if sum_table in list_deck:
                            no_plays.append(card)
                        else:
                            possible_plays.append(card)
                            subsets.append(subset)
                            sums = []
                            for value_sum in subsets:
                                sub = []
                                for single_card in value_sum:
                                    for pick in table:
                                        if single_card == pick[0]:
                                            single_card = pick
                                            sub.append(single_card)
                                sums.append(sub)
                            possible_picks[card] = sums
                    else:
                        copy = list_table[:]
                        copy.append(card[0])
                        sum_table = sum(rank_table for rank_table in copy)
                        if sum_table in list_deck:
                            pass
                        else:
                            possible_plays.append(card)
    possible_plays = list(set(possible_plays))
    for i in no_plays:
        if i in possible_plays:
            possible_plays.remove(i)
    return possible_picks, possible_plays

# Third def: BestMove

In case there are plays that the player can make without giving the opponent the possibility to make a Scopa, the 'AvoidScopa' function stores all these possible plays in a dictionary, made by the cards being played as keys and the picks as values (a list in case the pick is made by multiple cards, a list of lists if there are multiple combinations of picks).
Iterating for every key and value of the dictionary, the 'BestMove' function sums the value of the card that would be played and the value of the card/s that would be taken, returning the card to play that maximizes the final result.

In [6]:
def BestMove(possible_picks, possible_plays, legalMoves, Standalone, values = values):
    
    new_p2 = {}
    risultato = []
    punteggi_di_ciascuna_value = []

    # in the next 'if' loop we see if possible_picks is empty: if it isn't then we see among all 
    # the possible tuples which one is the one that allows us to get more points and we create a 
    # new dictionary p2 in which we save only (combined with each key) the combination that allows 
    # us to score more points.

    if len(possible_picks) != 0:
        key_list = list(possible_picks.keys())
        val_list = list(possible_picks.values())
        
        for i in range(len(possible_picks)):
            key = (key_list[i])
            val = (val_list[i])

            
            if isinstance(val,tuple):
                new_p2[key] = val

            else:
                # A brief example to show what happen in each cicle:
                #[[(3, 1), (5, 3)], [(1, 1), (3, 1), (4, 1)]]
                values_of_each_el = []
                for el in val:
                    # [(3, 1), (5, 3)]
                    somma = 0
                    for tup in el:
                        # (3, 1)
                        somma = somma + values[tup]
                        # 'somma' represents all points gained by taking that set of cards
                    values_of_each_el.append(somma) #values_of_each_el in this case it contains: [38, 73]

                massimo = max(values_of_each_el) 
                punteggi_di_ciascuna_value.append(massimo)
                index = values_of_each_el.index(massimo)
                new_p2[key] = val[index]
        
        key_list = list(new_p2.keys())
        val_list = list(new_p2.values())
        punteggi = []

        # here I calculate for each card how many points it takes
        for i in range(len(possible_picks)):
            key = (key_list[i])
            val = (val_list[i])
        
            punti_della_key = int(values[key])
            somma = 0
            if isinstance(val,tuple):
                punti_della_value = values[val]
            else:
                for el in val:
                    somma = somma + values[el]
                punti_della_value = somma
        
        
            punti_presi = punti_della_key + punti_della_value
    
            punteggi.append(punti_presi)
        
        massimo = max(punteggi)
        index = punteggi.index(massimo)
        # creation of the output
        risultato = (key_list[index])
        key=key_list[index]
        val=val_list[index]

        if Standalone:
            return risultato
        else:
            return {
                key: val
            }

                
    
    else:
        # if possible_plays is not empty, we play the best card in possible_plays
        # otherwise we look among all the cards in hand and choose the best one
        if len(possible_plays) != 0:
            min_score = float('inf')
            for choice in possible_plays:
                value = values[choice]  
                if value < min_score: 
                    min_score = value  
                    risultato = choice  

            
        else:
            min_score = float('inf')
            for choice in legalMoves:
                value = values[choice]  
                if value < min_score:  
                    min_score = value  
                    risultato = choice 


    if Standalone:
            return risultato
    else:
            return {
                risultato: ()
            }


# Finally: The Intermediate

The Greedy and the Intermediate Algorithms are Rule Based, meaning that they give an answer based on some specific cases. They take as input the cards the player has in their hand and the cards on the table. The hand and the table are made of two lists of tuples, representing the cards, each consisting of two numbers: (rank, suit).
The value of the card is defined thanks to a dictionary of values, which helps the Rule Based Algorithm to decide what to do.
As output of this algorithms, we can have: or just the tuple representing the card that the player is going to play, or a dictionary containing the card to play and the card/s to pick from the table.
Let's dive into the algorithm.

In [7]:
def Intermediate(legalMoves, table, deck, Standalone, values = values):
    if CheckForScopa(legalMoves, table, Standalone, values):
        return CheckForScopa(legalMoves, table, Standalone, values)
    else:
        return BestMove(AvoidScopa(legalMoves, table, deck)[0], AvoidScopa(legalMoves, table, deck)[1], legalMoves, Standalone, values)



# Some tests and examples

In [8]:
#############################################################

### EXAMPLE 1 ###

a = [(1, 2), (7, 1), (8, 2)] # cards in hand: `legalMoves`

b = [(1, 1), (3, 1), (4, 1), (7, 2), (5, 3)] # cards on the table: `table`

c = [] # cards not seen yet: `deck`


print(Intermediate(a, b, c, Standalone=False))

###############################################################

{(7, 1): (7, 1)}


In [10]:
#############################################################

### EXAMPLE 2: DIFFERENCE FROM THE GREEDY ###

a = [(1, 2),(7, 3),(9, 1)] # cards in hand: `legalMoves`

b = [(4, 1), (6, 1), (5, 3)] # cards on the table: `table`

c = [(6, 4), (3, 1), (4, 3)] # cards not seen yet: `deck`


print(Intermediate(a, b, c, Standalone=False))

###############################################################

{(1, 2): ()}
